In [7]:
"""
RAG
- Stuff Documents 체인을 사용하여 완전한 RAG 파이프라인을 구현하세요.
- 체인을 수동으로 구현해야 합니다.
- 체인에 `ConversationBufferMemory`를 부여합니다.
- 이 문서를 사용하여 RAG를 수행하세요: files/chapter_one.txt
- 체인에 다음 질문을 합니다:
    - Aaronson 은 유죄인가요?
    - 그가 테이블에 어떤 메시지를 썼나요?
    - Julia 는 누구인가요?
"""

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

# LLM 초기화 부분 수정
llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-3.5-turbo",
    max_tokens=1000  # 추가
)

# 스플리터 설정 수정
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\\n",
    chunk_size=200,  # 300 -> 200
    chunk_overlap=20,  # 30 -> 20
)
loader = UnstructuredFileLoader("../files/chapter_one.txt")
docs = loader.load_and_split(text_splitter=splitter)

# 임베딩 설정
cache_dir = LocalFileStore("./.cache/")
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# Vector Store 생성
vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever(
    search_kwargs={
        "k": 1,  # 2 -> 1
        "fetch_k": 2  # 추가
    }
)

# 메모리 설정
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history",
    input_key="question",
    output_key="answer",
)

# 프롬프트 템플릿 정의 - 더 간단하게
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "Answer based on the context. If unsure, say 'I don't know'.\nContext: {context}\nHistory: {chat_history}"
    ),
    ("human", "{question}"),
])

# Chain 구성
chain = {
    "context": retriever,
    "chat_history": lambda x: memory.load_memory_variables({})["chat_history"],
    "question": RunnablePassthrough(),
} | prompt | llm

# 질문 함수 정의
def ask_question(question):
    response = chain.invoke(question)
    memory.save_context(
        {"question": question},
        {"answer": response.content}
    )
    return response.content

# 테스트 질문
questions = [
    "Aaronson 은 유죄인가요?",
    "그가 테이블에 어떤 메시지를 썼나요?",
    "Julia 는 누구인가요?"
]

# 질문 실행
print("=== RAG 테스트 시작 ===\n")
for question in questions:
    print(f"질문: {question}")
    answer = ask_question(question)
    print(f"답변: {answer}\n")

=== RAG 테스트 시작 ===

질문: Aaronson 은 유죄인가요?
답변: Aaronson은 유죄로 여겨졌습니다. 이 문맥에서 Aaronson은 범죄로 인해 처벌받은 것으로 나타납니다.

질문: 그가 테이블에 어떤 메시지를 썼나요?
답변: 그가 테이블에 쓴 메시지는 "2+2=5" 였습니다.

질문: Julia 는 누구인가요?


InvalidRequestError: This model's maximum context length is 16385 tokens. However, you requested 16430 tokens (15430 in the messages, 1000 in the completion). Please reduce the length of the messages or completion.